# Módulo 07 - Tarefa 02

#### 1) Carregue a base e garanta que a base está como deveria.

Considere a base que você ajustou na lição passada. Carregue-a. Caso ainda haja alguma pendência, ajuste - lembre-se de que o scikitlearn não recebe variáveis em formato string, somente numéricas, e não aceita '*missings*'. 

- Separe 70% da base para treinamento e 30% para validação. Cada uma dessas partes terá dois objetos, um armazenando a variável resposta ```mau```e outro armazenando as variáveis explicativas (lembrando: sem variáveis string, já com as dummies).

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# 1.1 Carregando a base
df = pd.read_csv('demo01.csv')

# 1.2 Pré-processamento: Convertendo categóricas em dummies e removendo strings
df_encoded = pd.get_dummies(df, drop_first=True)

# 1.3 Separando Variável Resposta (y) e Explicativas (X)
y = df_encoded.mau
X = df_encoded.drop('mau', axis=1)

# 1.4 Separação Treino (70%) e Teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

print(f'Formato treino: {X_train.shape}')
print(f'Formato teste: {X_test.shape}')

#### 2) Vamos para o modelo:

1. Defina um objeto com a função da árvore de decisão vista em aula.
2. Treine o modelo com os dados que você separou para treinamento.
3. Visualize a árvore. Talvez você queira aumentar um pouco a figura.
4. Produza uma visualização da matriz de classificação (ou matriz de confusão) - coloque os rótulos como "aprovados" e "reprovados" - pois afinal de contas, são essas as decisões que se podem tomar com propostas de crédito.
5. Calcule a acurácia na base de treinamento

In [ ]:
# 2.1 e 2.2 Definindo e treinando o modelo
clf = DecisionTreeClassifier(random_state=123)
clf.fit(X_train, y_train)

# 2.3 Visualizando a árvore
plt.figure(figsize=(25, 10))
plot_tree(clf, 
          filled=True, 
          class_names=['Aprovados', 'Reprovados'], 
          feature_names=list(X.columns), 
          max_depth=3) # Limitado visualmente para clareza
plt.show()

# 2.4 Matriz de Confusão (Treino)
y_pred_train = clf.predict(X_train)
cm_train = confusion_matrix(y_train, y_pred_train)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=['Aprovados', 'Reprovados'])
disp.plot(cmap='Blues')
plt.title('Matriz de Confusão - Treino')
plt.show()

# 2.5 Acurácia Treino
print(f'Acurácia Treino: {accuracy_score(y_train, y_pred_train):.4f}')

#### 3) Vamos avaliar o modelo na base de testes

1. Classifique a base de teste de acordo com a árvore que você treinou no item 2.
2. Produza a visualização da matriz de confusão para a base de teste.
3. Calcule a acurácia da base de teste. Compare com a acurácia da base de treinamento.
4. Treine uma nova árvore com número mínimo de observações por folha de 5 e máximo de profundidade de 10. Use o random_state = 123. Avalie a matriz de classificação. Observe a distribuição da predição - qual a proporção de proponentes foram classificados como 'maus'?
5. Como ficaria a acurácia se você classificasse todos os contratos como 'bons'?

In [ ]:
# 3.1 Classificação na base de teste
y_pred_test = clf.predict(X_test)

# 3.2 Matriz de Confusão (Teste)
cm_test = confusion_matrix(y_test, y_pred_test)
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=['Aprovados', 'Reprovados'])
disp_test.plot(cmap='Greens')
plt.title('Matriz de Confusão - Teste')
plt.show()

# 3.3 Comparação de Acurácia
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
print(f'Acurácia Treino: {acc_train:.4f}')
print(f'Acurácia Teste:  {acc_test:.4f}')

# 3.4 Nova Árvore (Podada)
clf_podada = DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, random_state=123)
clf_podada.fit(X_train, y_train)

y_pred_podada = clf_podada.predict(X_test)
prop_maus = y_pred_podada.sum() / len(y_pred_podada)

print(f'\nAcurácia Nova Árvore: {accuracy_score(y_test, y_pred_podada):.4f}')
print(f'Proporção de proponentes "Maus": {prop_maus:.2%}')

# 3.5 Acurácia classificando todos como 'bons'
y_all_bons = [0] * len(y_test)
print(f'Acurácia se todos fossem "Bons": {accuracy_score(y_test, y_all_bons):.4f}')